In [1]:
import pdb
import json
import pickle
import argparse
import pandas as pd
import networkx as nx
from collections import Counter
# import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# sns.set(font_scale=1.5)
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from google.colab import drive
drive.mount('/content/drive')

# 安装gensim: 
# !pip install gensim
# from gensim.models import KeyedVectors
# FILE="/content/drive/MyDrive/code/bert_related/wiki-news-300d-1M.vec"
# FASTTEXT=KeyedVectors.load_word2vec_format(FILE,limit=500000)

# 安装sentence-transformers:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

# 加载roberta-large-nli-stsb-mean-tokens
# model = SentenceTransformer("araphrase-MiniLM-L6-v2")
sentence_model = SentenceTransformer("roberta-large-nli-stsb-mean-tokens")

# 安装huggingface transformers（版本大于4.3.0）:
# !pip install transformers
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") 
# model = AutoModel.from_pretrained("bert-base-cased")




Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=2ca0344de653c996130314cb631422278eedbe251c4a6e20f87ecc139ba4f1b9
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [2]:
data = pd.read_csv("/content/drive/MyDrive/code/bert_related/labeled_data.csv", index_col = False)
data = data[data['agreement_fraction']==1.000000]
bb = data[["author_parent","author_child",'label','datetime','subreddit','body_parent', 'body_child']]
bb_sub = data[['body_parent', 'body_child']]

In [9]:
bb_sub[0:100].to_csv('sample_data.csv')

In [10]:
par_text_list = list(bb_sub['body_parent'])
chi_text_list = list(bb_sub['body_child'])
len(par_text_list)

16723

In [13]:
par_text_list[1]

'Tlaib, your family is Hamas. Funded by Iran a state sponsor of terrorism. Hamas is a terrorist organization 3000 rockets shot at innocent civilians. All you do is speak negatively about the US. You are solely in power for evil ways and means. I think your a really shitty politician that is anti-American.'

In [14]:
def emb_from_word2vec(par_text_list):
  par_word2vec = []
  for i in range(len(par_text_list)):
    par_text_split = par_text_list[i].split(' ')
    pdb.set_trace()
    tmp_par = []
    for j in range(len(par_text_split)):
      print(j, par_text_split[j])
      tmp_par.append(FASTTEXT[par_text_split[j]])
    pdb.set_trace()
    df_tmp_par = pd.DataFrame(tmp_par)
    tmp_par = df_tmp_par.sum(axis=1)/len(par_text_split)  
    par_word2vec.append(tmp_par)
  return par_word2vec

In [ ]:
# use word embeddings as sentence embedding

par_w2v = emb_from_word2vec(par_text_list[0:2])
chi_w2v = emb_from_word2vec(chi_text_list[0:2])


par_word2vec = []
for i in range(len(par_text_list[0:2])):
  par_text_split = par_text_list[i].split(' ')
  tmp_par = []
  for j in range(len(par_text_split)):
    print(j, par_text_split[j])
    tmp_par.append(FASTTEXT[par_text_split[j]])




In [15]:
bb = pd.read_csv('/content/drive/MyDrive/code/bert_related/sentence_parent.csv',index_col=[0])


FileNotFoundError: ignored

In [ ]:
# sentence model for par
import numpy as np
par_output = []

for i in range(1,len(par_text_list)):
  pdb.set_trace()
  print(par_text_list[i])
  per_par_out = sentence_model.encode(par_text_list[i])
  par_output.append(per_par_out)



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.9/bdb.py", line 334, in set_trace
    sys.settrace(self.trace_dispatch)



> <ipython-input-19-f52faab41885>(7)<module>()
-> print(par_text_list[i])


In [ ]:
# preserve sentence model for par

len(par_output)
aa = pd.DataFrame(par_output)
aa.to_csv('/content/drive/MyDrive/code/bert_related/sentence_parent.csv')
# bb = pd.read_csv('sentence_ber_par.csv',index_col=[0])

In [ ]:
# add par0 
par0 = sentence_model.encode(par_text_list[0])
par1 = bb
df_p0 = pd.DataFrame(par0).T

# print(df_p0)
# print(bb[0:1])
print(df_p0.shape, par1.shape)

# cc = df_p0.join(par1)
# print(cc.shape, cc)

df_p0.columns = par1.columns
print(df_p0.columns)
print(par1.columns)

cc = pd.concat([df_p0,par1],ignore_index=True)

print(cc.shape, cc)
cc.to_csv('/content/drive/MyDrive/code/bert_related/new_sentence_parent.csv')